In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns; sns.set()
import warnings
import apyori
warnings.simplefilter(action='ignore', category=FutureWarning)


In [3]:
# import data
orders = pd.read_csv("orders.csv")
train = pd.read_csv("order_products__train.csv")
products = pd.read_csv("products.csv")

In [4]:
# using train as prior will not fit into memory
train_orders = train.merge(orders, on = 'order_id', how = 'left')
train_orders = train_orders.merge(products, on = 'product_id', how = 'left')

In [6]:
def memory_cruncher(df):
    """Reduces memory usage of data frame by converting the columns in to smaller dtypes"""
    for c in df.columns:
        if df[c].dtype == 'float64':
            df[c] = df[c].astype('float32')
        elif df[c].dtype == 'int64':
            df[c] = df[c].astype('int32')
        elif c == 'reordered':
            df[c] == df[c].astype('int8')
        else:
            pass
    return df
memory_cruncher(train_orders)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16
1,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16
2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4
3,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4
4,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,train,30,0,10,4.0,Natural Artesian Water,115,7
1384613,3421063,35548,4,1,169679,train,30,0,10,4.0,Twice Baked Potatoes,13,20
1384614,3421070,35951,1,1,139822,train,15,6,10,8.0,Organic Unsweetened Almond Milk,91,16
1384615,3421070,16953,2,1,139822,train,15,6,10,8.0,Creamy Peanut Butter,88,13


In [7]:
train_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 13 columns):
order_id                  1384617 non-null int32
product_id                1384617 non-null int32
add_to_cart_order         1384617 non-null int32
reordered                 1384617 non-null int32
user_id                   1384617 non-null int32
eval_set                  1384617 non-null object
order_number              1384617 non-null int32
order_dow                 1384617 non-null int32
order_hour_of_day         1384617 non-null int32
days_since_prior_order    1384617 non-null float32
product_name              1384617 non-null object
aisle_id                  1384617 non-null int32
department_id             1384617 non-null int32
dtypes: float32(1), int32(10), object(2)
memory usage: 89.8+ MB


In [9]:
# new df for order_id and product_id to be used for dummy variables 
df_ap = train_orders[['order_id', 'product_name']]

In [11]:
del train_orders

In [10]:
# check for null values
df_ap.isnull().sum()

order_id        0
product_name    0
dtype: int64

In [13]:
# creating a dictionary with all of the products in one list for each order
dict_ap = df_ap.groupby('order_id').product_name.agg(list).to_dict()

In [14]:
# nested lists format for input into algorithm
transactions = []
for order_id, transaction in dict_ap.items():
    transactions.append(str(transaction))

In [15]:
# items sets obtained from apriori with a minumum support of .6 can use filtering later to increase support
frequent_itemsets = list(apyori.apriori(
                                    transactions,
                                    min_support=0.05,
                                    min_confidence=0.80,
                                    min_lift=1.0,
                                    max_length=None
                                   ))

KeyboardInterrupt: 

In [ ]:
# analyze relation records
frequent_itemsets